In [ ]:
# Import dependencies
import pandas as pd
import sqlalchemy
from pprint import pprint
from flask import Flask, jsonify
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func

In [ ]:
# Create engine to wardata.sqlite
engine = create_engine("sqlite:///data/wardata.sqlite")
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [ ]:
Base = declarative_base()
class Baseball(Base):
    __tablename__ = "baseball"
    player_id = Column(Integer, primary_key=True)
    rank_in_country = Column(Integer)
    player_name = Column(String)
    war = Column(Float)
    year_min = Column(Integer)
    year_max = Column(Integer)
    age_range = Column(String)
    birth_location = Column(String)
    country = Column(String)
    position = Column(String)

In [ ]:
variable = session.query(Baseball)

#Sample Query
new_df = pd.read_sql("SELECT * FROM baseball WHERE player_name LIKE 'Babe Ruth'", engine)
new_df